In [ ]:
import numpy as np
import pandas as pd
import yaml
import seaborn as sns
import matplotlib.pyplot as plt

from src.utils.general import get_db_conn, get_s3_credentials

In [ ]:
conn = get_db_conn("../config/local/credentials.yaml")

# Limpieza

**Para ver 200 filas en el notebook**

In [ ]:
pd.set_option('display.max_rows', 2000)

**Para ver 10 columnas en el notebook**

In [ ]:
pd.set_option('display.max_columns', 10)

**Query para obtener la base de datos**

In [ ]:
 # Query
q = """
    select 
        *
    from 
        raw.unidades_economicas_denue
        
    limit 100000 
    
    -- Comentario: limite para solo obtener las primeras 100000 observaciones
"""

In [ ]:
# Datos
df = pd.read_sql(q, conn)

## 1. ¿Cuántas variables y observaciones tenemos?

In [ ]:
df.shape

In [ ]:
# Renglones
df.shape[0]

In [ ]:
# Columnas
df.shape[1]

In [ ]:
# Nombre de columnas
df.columns

In [ ]:
# Las primeras 5 observaciones
df.head(5)

In [ ]:
# Las últimas 5 observaciones
df.tail(5)

In [ ]:
# Muestra aleatoria de 5 observaciones, con semilla para que sea reproducible.
df.sample(5,random_state=123)

In [ ]:
# Muestra aletoria del 10% de las observaciones con semilla para que sea reproducible.
df.sample(frac=0.10, random_state=123)

## 2. Tipos de datos por columna

In [ ]:
df.dtypes

### Cambiar tipo de datos

#### Hacer variable de tipo carácter (str)

In [ ]:
df['id'] = df['id'].astype('str')

#### Hacer variable de tipo numérica: entero (int) o flotante (float64)

In [ ]:
df['id'].astype('int')

In [ ]:
df['id'].astype('float64')

In [ ]:
# Otra manera de hacer la variable numérica cuando salen errores en las otras funciones
df['num_exterior'] = pd.to_numeric(df['num_exterior'])

#### Hacer variable de tipo temporal

In [ ]:
df['date_retrieved'] = pd.to_datetime(df['date_retrieved'])

### 3. Variables categorícas

In [ ]:
# Valores únicos por variable. En las variables string aparecerán el número de categorías únicas
df.nunique()

**Si queremos evaluar más a detalle una variable categórica:**

In [ ]:
# Categorías en variable
df['sector'].unique()

In [ ]:
#  Vemos el número de observaciones por categoría de esta variable
conteos = df['sector'].value_counts()
conteos

In [ ]:
# Proporción de categorías
prop = df['sector'].value_counts(normalize=True)
prop

In [ ]:
# Lo visualizamos mejor:
pd.concat([conteos,prop], 
          keys=['num','prop'], 
          axis=1)

### 4. Variables numéricas

In [ ]:
# Todas las variables que son numéricas las describe con esas características.
df.describe()

### 5. Variables temporales

In [ ]:
# Para obtener el mes de la variable que previamente ya habíamos hecho temporal
df['mes']=df['date_retrieved'].dt.month

In [ ]:
# Año
df['date_retrieved'].dt.year

### 6. Ver número de NA's por columna

In [ ]:
df.isna().sum()

### Algunos trucos extra

Quitar columnas

In [ ]:
df.drop(['clee',
         'razon_social',
         'num_interior'], 
        axis=1, # 0 es por filas y 1 por columnas.
        inplace=True # True es para guardar los cambios sobre la misma base. 
       )

#### Seleccionando subconjuntos de la base

`.loc` vs `.iloc`

- `.loc`: utiliza las etiquetas de los índices y de las columnas, es decir el nombre como un string.

- `iloc`: utiliza la posición como entero del dataframe.

Seleccionamos todas las filas (`:`), de las columnas que están desde la posición 0 hasta la 3, y luego las columnas 4, 8, 10.

In [ ]:
df.iloc[:,0:3]

In [ ]:
df.iloc[:,[0,2,4]]

In [ ]:
# Selecciona las filas 2 hasta la 6
df.iloc[2:6,]

In [ ]:
# Selecciona filas con los índices 122, 126 y 208:
df.iloc[[122,126,208],]

Seleccionar columnas de acuerdo a las etiquetas de las columnas

In [ ]:
df.loc[:,'sector':'rama']

In [ ]:
df.loc[:,['localidad','rama','clase']]

In [ ]:
df.loc[0:1,]

## Selecionar filas basadas en condiciones

In [ ]:
df[(['num_exterior','clase','rama'])].query('5 > num_exterior')

### Para buscar las filas en las que ciertas categorías aparecen (pueden ser tantas categorías como se necesite).

In [ ]:
df['tipo_vialidad'].unique()

In [ ]:
# seleccionar solo las categorias que tienen corredor y autopista
categories_interes = ['CORREDOR', 'AUTOPISTA']
df[df['tipo_vialidad'].isin(categories_interes)]

In [ ]:
df2 = df[df['nombre'].str.contains('FARMA')]

In [ ]:
# nueva columna con el index
df2.reset_index().head()

In [ ]:
# Hace un drop del index
df2.reset_index(drop=True).head()

In [ ]:
# Quita el index sobre la base de datos df2
df2.reset_index(inplace=True)

In [ ]:
df2.head()

# Listas y arreglos

Las bases de datos se trabajan generalmente con la paquetería pandas (pd), sin embargo, existe mucha información que está desplegada de otras maneras y para utilizarla necesitas numpy (np) u otras paqueterías, ya cargadas al inicio del notebook. 

- listas
- arreglos

In [ ]:
# arreglo de repetir ccd 4 veces.
np.repeat('ccd',4)

In [ ]:
# lista de repetir ccd 4 veces
list(np.repeat('ccd',4))

In [ ]:
hola = list(np.array([8,2,9,25,4]))
hola

In [ ]:
adios = list(np.array([66,9,17,30,5]))
adios

Podemos hacer operaciones con arreglos y con listas, una manera de hacerlo sería con la siguiente.

In [ ]:
# Resta
buen_dia = list(map(lambda x,y: x-y, adios, hola))
buen_dia

Para agregar elementos a una lista de manera iterativa. Puede ser como un conjunto de listas, o una lista con todos los elementos:

In [ ]:
listas_de_listas = []

In [ ]:
# Aquí utilizamos append
for i in range(0,len(buen_dia)):
    listas_de_listas.append(np.repeat(hola[i],buen_dia[i]))

In [ ]:
listas_de_listas

In [ ]:
lista_de_listas = []

In [ ]:
# Aquí utilizamos extend
for i in range(0,len(buen_dia)):
    lista_de_listas.extend(np.repeat(hola[i],buen_dia[i]))

In [ ]:
lista_de_listas